In [9]:
# Import modules
import boto3
import csv
import json

In [11]:
# Create a Bedrock client using boto3 to interact with AWS Bedrock's runtime service
bedrock_client = boto3.client('bedrock-runtime')

In [12]:
# List of all holdings files to process
holdings_files = [
    "oct_holdings_2024.csv",
    "nov_holdings_2024.csv"
]

# Converts CSV file content into a string with pipe separators between values
def format_holdings(filename):
    with open(filename, mode="r", newline="", encoding="utf-8") as file:
        csv_reader = csv.reader(file)
        return "\n".join(" | ".join(row) for row in csv_reader)

# Process all files in one line
formatted_holdings = {filename: format_holdings(filename) for filename in holdings_files}

In [13]:
formatted_holdings["oct_holdings_2024.csv"]

"\ufeffHoldings details\n\nS&P 500 Growth ETF (VOOG)\n\n\nEquity | as of 10/31/2024\n\n | SEDOL | HOLDINGS | TICKER | % OF FUNDS* | SECTOR | COUNTRY | SECURITYDEPOSITORYRECEIPTTYPE | MARKET VALUE* | SHARES\n | 2046251 | Apple Inc | AAPL | 12.48% | Technology Hardware, Storage & Peripherals | US | --- | $1,648,462,333.17 | 7,296,987\n | 2379504 | NVIDIA Corp | NVDA | 11.87% | Semiconductors | US | --- | $1,567,422,045.08 | 11,806,433\n | 2588173 | Microsoft Corp | MSFT | 10.98% | Systems Software | US | --- | $1,449,635,339.25 | 3,567,455\n | 2000019 | Amazon.com Inc | AMZN | 6.33% | Broadline Retail | US | --- | $835,669,971.20 | 4,483,208\n | B7TL820 | Meta Platforms Inc | META | 4.51% | Interactive Media & Services | US | --- | $595,135,441.42 | 1,048,549\n | BYVY8G0 | Alphabet Inc | GOOGL | 3.64% | Interactive Media & Services | US | --- | $481,161,148.89 | 2,811,999\n | BYY88Y7 | Alphabet Inc | GOOG | 3.01% | Interactive Media & Services | US | --- | $398,081,361.51 | 2,305,179\n |

In [14]:
formatted_holdings["nov_holdings_2024.csv"]

"\ufeffHoldings details\n\nS&P 500 Growth ETF (VOOG)\n\n\nEquity | as of 11/30/2024\n\n | SEDOL | HOLDINGS | TICKER | % OF FUNDS* | SECTOR | COUNTRY | SECURITYDEPOSITORYRECEIPTTYPE | MARKET VALUE* | SHARES\n | 2046251 | Apple Inc | AAPL | 12.38% | Technology Hardware, Storage & Peripherals | US | --- | $1,778,514,233.85 | 7,493,845\n | 2379504 | NVIDIA Corp | NVDA | 11.67% | Semiconductors | US | --- | $1,676,272,678.50 | 12,124,938\n | 2588173 | Microsoft Corp | MSFT | 10.80% | Systems Software | US | --- | $1,551,393,137.52 | 3,663,612\n | 2000019 | Amazon.com Inc | AMZN | 6.66% | Broadline Retail | US | --- | $957,126,807.34 | 4,604,006\n | B7TL820 | Meta Platforms Inc | META | 4.31% | Interactive Media & Services | US | --- | $618,434,667.84 | 1,076,812\n | BYVY8G0 | Alphabet Inc | GOOGL | 3.40% | Interactive Media & Services | US | --- | $487,873,536.00 | 2,887,680\n | B616C79 | Tesla Inc | TSLA | 3.29% | Automobile Manufacturers | US | --- | $472,021,141.88 | 1,367,543\n | BYY88Y

In [15]:
# Construct the prompt configuration using the JSON string
prompt_config = {
    "anthropic_version": "bedrock-2023-05-31",
    "temperature": 1,
    "top_p": 0.999,
    "top_k": 250,
    "max_tokens": 2000, 
    "stop_sequences": ["\n\nHuman:"], 
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "I have provided you with two documents named formatted_jan_holding and formatted_feb_holding. Please compare these two documents and create a list of the top 30 holdings whose share count has changed between January and February. For each holding, please only provide the holdings name, ticker symbol and the difference in shares between January and February.:"},
                {"type": "text", "text": formatted_holdings["oct_holdings_2024.csv"]}, 
                {"type": "text", "text": formatted_holdings["nov_holdings_2024.csv"]},
            ],
        }
    ],
}

In [16]:
# Prepare the request configuration for an AI model API call
body = json.dumps(prompt_config)
contentType = "application/json"
accept = "application/json"
modelId = "anthropic.claude-3-haiku-20240307-v1:0"

In [17]:
# Invokes the specified Bedrock model
response = bedrock_client.invoke_model(
    body=body, 
    contentType=contentType,
    accept=accept,
    modelId=modelId,  
)

In [18]:
# Parse the JSON response body from a bytes/string format into a Python dictionary/object
response_body = json.loads(response.get("body").read())

In [19]:
# Extract the text content from the first item in the 'content' array of the response
results = response_body.get("content")[0].get("text")

In [20]:
results

'Here is the list of the top 30 holdings whose share count has changed between January and February:\n\n1. Apple Inc (AAPL) - Increase of 196,858 shares\n2. NVIDIA Corp (NVDA) - Increase of 318,505 shares \n3. Microsoft Corp (MSFT) - Increase of 96,157 shares\n4. Amazon.com Inc (AMZN) - Increase of 120,798 shares\n5. Meta Platforms Inc (META) - Increase of 28,263 shares\n6. Alphabet Inc (GOOGL) - Increase of 75,681 shares\n7. Tesla Inc (TSLA) - Increase of 35,897 shares\n8. Alphabet Inc (GOOG) - Increase of 62,273 shares\n9. Broadcom Inc (AVGO) - Increase of 60,234 shares\n10. Eli Lilly & Co (LLY) - Increase of 10,213 shares\n11. Netflix Inc (NFLX) - Increase of 5,556 shares\n12. Visa Inc (V) - Increase of 14,218 shares\n13. Salesforce Inc (CRM) - Increase of 12,523 shares\n14. Mastercard Inc (MA) - Increase of 7,655 shares\n15. Oracle Corp (ORCL) - Increase of 20,620 shares\n16. Adobe Inc (ADBE) - Increase of 5,730 shares\n17. Advanced Micro Devices Inc (AMD) - Increase of 20,900 shar

In [21]:
# Specify the filename you want to write to
filename = "findings.txt"

In [22]:
# Open the file in write mode ('w') and write the results to it
with open(filename, "w") as file:
    file.write(results)